In [3]:
import pickle
#from loadData import dataLoader
import tensorflow as tf
from CalculateEER import SKGetEER as GetEER
import numpy as np
import xgboost as xgb

In [4]:
def onehot(x,numlabels):
    t = [0 for i in range(numlabels)]
    t[x-1] = 1
    return t

In [7]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
data= pickle.load(open('/Users/chaitanya/Documents/python/keystrokes/data_augmentation/greyc1/PCA_GREYC_2X.pickle','rb'))


In [10]:
X = data['data']
X = np.array(X).reshape(len(X), 28)
y = data['labels']
#y_train = [onehot(x,100) for x in train_data['labels']]
#X_test = np.array(test_data['features']).reshape(len(test_data['features']), 1, 28, 1)
#y_test = [onehot(x,100) for x in test_data['labels']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
#X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=122)
print([len(x) for x in [X_test, y_test, X_train, y_train]])
test_Y = y_test


[4445, 4445, 10371, 10371]


In [19]:
xg_train = xgb.DMatrix(X_train, label=  y_train)
xg_test = xgb.DMatrix(X_test, label= y_test)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softprob'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 10
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 100
param['eval_metric'] = 'auc'
n_classes = 100
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 10
bst = xgb.train(param, xg_train, num_round, watchlist)
# get prediction
pred = bst.predict(xg_test)
print(np.size(pred))
error_rate = np.sum(pred != test_Y) / np.shape(test_Y)[0]
print('Test error using softmax = {}'.format(1-error_rate))

# do the same thing again, but output probabilities
'''param['objective'] = 'multi:softprob'
bst = xgb.train(param, xg_train, num_round, watchlist)
# Note: this convention has been changed since xgboost-unity
# get prediction, this is in 1D array, need reshape to (ndata, nclass)
pred_prob = bst.predict(xg_test).reshape(np.shape(test_Y)[0], n_classes)
print(pred_prob)
pred_label = np.argmax(pred_prob, axis=1)
error_rate = np.sum(pred_label != test_Y) / np.shape(test_Y)[0]
print('Test error using softprob = {}'.format(1-error_rate))'''

XGBoostError: b'[12:06:34] src/metric/rank_metric.cc:88: Check failed: preds.size() == info.labels_.size() (1037100 vs. 10371) label size predict size not match\n\nStack trace returned 2 entries:\n[bt] (0) 0   libxgboost.dylib                    0x0000000119431683 dmlc::StackTrace[abi:cxx11]() + 67\n[bt] (1) 1   libstdc++.6.dylib                   0x00000001198b0ce0 vtable for std::__cxx11::basic_stringbuf<char, std::char_traits<char>, std::allocator<char> > + 16\n\n'

(?, 1, 7, 128)


20241